In [8]:
#pip install fix-yahoo-finance
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [10]:
#import list of CHstocks with yahoo tickers
large = pd.read_excel('CHtickers.xlsx', sheet_name='Large Cap')
mega = pd.read_excel('CHtickers.xlsx', sheet_name='Mega Cap')
tickers = pd.concat([large,mega], axis=0)
tickers = tickers[['Symbol','Name','Market Cap']]
tickers.head()

FileNotFoundError: [Errno 2] No such file or directory: 'CHtickers.xlsx'

In [ ]:
#to be able to run through, converting dataframe column to list
mega_list = mega['Symbol'].to_list()
large_list = large['Symbol'].to_list()
tickers_list = mega_list + large_list

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf

#OPTIONS for RETRIEVAL

#first define timeframe of retrieval
#start=datetime.datetime(2021, 3, 1)
#end=datetime.datetime(2021, 3, 18)

#OR 

# use "period" instead of start/end
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# (optional, default is '1mo')
period = "2d"

# fetch data by interval (including intraday if period < 60 days)
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# (optional, default is '1d')
#interval = "90m"

# group by ticker (to access via data['SPY'])
# (optional, default is 'column')
group_by = 'ticker'

#then get data on the list

stocks = yf.download(tickers_list, 
                          #start=start, end=end, 
                          period=period, #interval=interval,
                          #group_by=group_by
                            )
stocks.head()

In [3]:
#  yfinance returns a multi-level column names, with a level for the ticker and a level for the stock price data

df = stocks.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1).T
df.head()

NameError: name 'stocks' is not defined

In [ ]:
#dataframe formatting

df.columns = ['Yday Adj Close', 'Yday Close', 'Yday High', 'Yday Low', 'Yday Open','Yday Volume',
              'Today Adj Close', 'Today Close', 'Today High', 'Today Low', 'Today Open','Today Volume',
             'Today Adj Close2', 'Today Close2', 'Today High2', 'Today Low2', 'Today Open2','Today Volume2' ]
df = df.iloc[1: , :]
df = df[['Yday Close','Today Open']]
df = df.astype(float)
df.head()

In [ ]:
#adding a column CHANGE and sort by it 
df['change'] = (df['Today Open']-df['Yday Close'])/df['Yday Close'] *100

df = df.sort_values(by = 'change', ascending = False)
df = df[df['change'].notna()]
df.head(10)

In [ ]:
#getting only the extreams TOP 10 and BOTTOM 10
top = df.head(10)
bottom = df.tail(10)
extream = pd.concat([top, bottom], axis=0)
extream = extream.reset_index()
extream.head()

In [ ]:
#lookup for name / sector from dataframe excel
extreamz = pd.merge(extream, total, left_on='index', right_on='Symbol')
extreamz.head()

In [ ]:
#output in excel
extreamz.to_csv('CHmovers115open.csv', index = False)